In [127]:
import threading, logging, time
import concurrent.futures

class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        
        logging.info("Thread %s: starting update", name)
        self._lock.acquire()   # --- locking the code to one thread
        
        try:
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
        except:
            print("oops")
            
        finally:
            self._lock.release()   # --- release the code to one thread
        
        logging.info("Thread %s: finishing update", name)
        

In [128]:
if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    database = FakeDatabase()
    logging.info("Testing update. Starting value is %d.", database.value)
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for index in range(2):
            executor.submit(database.update, index)
    logging.info("Testing update. Ending value is %d.", database.value)

16:35:46: Testing update. Starting value is 0.
16:35:46: Thread 0: starting update
16:35:46: Thread 1: starting update
16:35:46: Thread 0: finishing update
16:35:46: Thread 1: finishing update
16:35:46: Testing update. Ending value is 2.
